In [125]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm

pd.set_option('max_columns',100)
%matplotlib inline
plt.rcParams['figure.figsize'] = (10,10)

In [3]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [143]:
df_train["MoSold"] = df_train["MoSold"].astype('category')
df_test["MoSold"] = df_test["MoSold"].astype('category')


In [144]:
y_train = df_train['SalePrice']
X_train = df_train.drop(['SalePrice','Id'], axis=1)
X_test = df_test.drop(['Id'], axis=1)


In [40]:
predict_vars1 = ['LotArea','GrLivArea','OverallQual','OverallCond','YearBuilt','YearRemodAdd','FullBath',\
                 'HalfBath','BedroomAbvGr','KitchenAbvGr','GarageCars','MoSold','YrSold',\
                 'Neighborhood','KitchenQual','ExterQual']
predict_vars2 = ['LotArea','GrLivArea','YearBuilt','FullBath',\
                 'BedroomAbvGr','KitchenAbvGr','GarageCars']
predict_vars3 = ['LotArea','GrLivArea','OverallQual','YearBuilt','FullBath','BedroomAbvGr']

In [147]:
df1 = df_train[predict_vars1]  
df2 = df_train[predict_vars2]
df3 = df_train[predict_vars3]

In [148]:
df1 = pd.get_dummies(df1, drop_first=True, dummy_na=False)
#consider switching MoSold, Qual & Cond variables to categorical?

In [149]:
df1.head()
# print(df2.head())
# print(df3.head())


,LotArea,GrLivArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,GarageCars,YrSold,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,ExterQual_Fa,ExterQual_Gd,ExterQual_TA
0,8450,1710,7,5,2003,2003,2,1,3,1,2,2008,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1,9600,1262,6,8,1976,1976,2,0,3,1,2,2007,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1
2,11250,1786,7,5,2001,2002,2,1,3,1,2,2008,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,9550,1717,7,5,1915,1970,1,0,3,1,3,2006,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4,14260,2198,8,5,2000,2000,2,1,4,1,3,2008,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [150]:
#Split Kaggle (training) data set into training & test sets (80/20)

seed = 0
x1_train,x1_test,y1_train,y1_test = train_test_split(df1,y_train,test_size=0.20,random_state=seed)
x2_train,x2_test,y2_train,y2_test = train_test_split(df2,y_train,test_size=0.20,random_state=seed)
x3_train,x3_test,y3_train,y3_test = train_test_split(df3,y_train,test_size=0.20,random_state=seed)


In [151]:
#LINEAR REGRESSION:  df1   

linear1 = linear_model.LinearRegression()
linear1.fit(x1_train, y1_train)
linear1.score(x1_train, y1_train)
print("Coefficients:", linear1.coef_)
print("Intercept:", linear1.intercept_)
print("R² Value:", linear1.score(x1_train, y1_train))
print("RSS:", np.sum((linear1.predict(x1_train) - y1_train) ** 2))

predicted1 = linear1.predict(x1_test)   # **** BEST result so far (R² = 0.86) 


Coefficients: [  7.09874922e-01   6.60010366e+01   1.07633143e+04   5.71905457e+03
   4.98777377e+02   5.84916724e+01  -1.30456961e+03  -8.18561791e+03
  -5.05193068e+03  -2.31178836e+04   1.01041520e+04  -3.42660600e+02
  -1.33382921e+04  -1.24704493e+04  -1.30000672e+04  -6.79881545e+03
  -1.28056283e+04  -9.91878226e+03  -1.32509084e+04  -1.03255481e+04
  -2.09035604e+04  -1.17948876e+04  -1.40142298e+04  -2.03172848e+04
  -1.52884957e+04   1.07736888e+04   2.07347218e+04   1.30736367e+04
   3.41870881e+04  -4.03171053e+01   1.72332098e+03  -1.09146111e+03
  -1.26771075e+04   6.72804995e+03   9.91562935e+03   7.06407755e+02
   4.89851963e+03   6.57949920e+04   4.44020364e+04  -1.77352070e+02
   2.32678040e+02   8.56946359e+03   5.90317263e+03   1.75121189e+04
   5.81785530e+04   1.91005677e+04   2.70817016e+04  -3.47597331e+04
  -3.62470923e+04  -3.87644715e+04  -3.36998103e+04  -3.79701284e+04
  -3.68755539e+04]
Intercept: -339615.117198
R² Value: 0.86051339255
RSS: 1003094154901.7

In [154]:
x = sm.add_constant(x1_train)
x1 = x1_train
model = sm.OLS(y1_train, x1)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.978
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                     930.6
Date:                Tue, 21 Aug 2018   Prob (F-statistic):               0.00
Time:                        21:48:17   Log-Likelihood:                -13671.
No. Observations:                1168   AIC:                         2.745e+04
Df Residuals:                    1115   BIC:                         2.772e+04
Df Model:                          53                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
LotArea                  0.7095 

In [160]:
y1_train.head()


618    314813
870    109500
92     163500
817    271000
302    205000
Name: SalePrice, dtype: int64

In [162]:
x1 = x1_train
y = np.log(y1_train)
model = sm.OLS(y, x1)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.566e+05
Date:                Tue, 21 Aug 2018   Prob (F-statistic):               0.00
Time:                        21:54:00   Log-Likelihood:                 643.89
No. Observations:                1168   AIC:                            -1182.
Df Residuals:                    1115   BIC:                            -913.4
Df Model:                          53                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
LotArea               2.931e-06 

In [139]:
#LINEAR REGRESSION:  df2

linear2 = linear_model.LinearRegression()
linear2.fit(x2_train, y2_train)
linear2.score(x2_train, y2_train)
print("Coefficients:", linear2.coef_)
print("Intercept:", linear2.intercept_)
print("R² Value:", linear2.score(x2_train, y2_train))
print("RSS:", np.sum((linear2.predict(x2_train) - y2_train) ** 2))
predicted2 = linear2.predict(x2_test)

Coefficients: [  7.12089008e-01   9.93841651e+01   6.90789717e+02   2.39271610e+02
  -1.51265795e+04  -4.19445360e+04   2.02266998e+04]
Intercept: -1287942.75136
R² Value: 0.746485277235
RSS: 1823107904309.4976


In [140]:
x = sm.add_constant(x2_train)
model = sm.OLS(y2_train, x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.746
Model:                            OLS   Adj. R-squared:                  0.745
Method:                 Least Squares   F-statistic:                     488.0
Date:                Tue, 21 Aug 2018   Prob (F-statistic):               0.00
Time:                        16:28:51   Log-Likelihood:                -14020.
No. Observations:                1168   AIC:                         2.806e+04
Df Residuals:                    1160   BIC:                         2.810e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.288e+06   1.01e+05    -12.706   

In [141]:
#LINEAR REGRESSION:  df3

linear3 = linear_model.LinearRegression()
linear3.fit(x3_train, y3_train)
linear3.score(x3_train, y3_train)
print("Coefficients:", linear3.coef_)
print("Intercept:", linear3.intercept_)
print("R² Value:", linear3.score(x3_train, y3_train))
print("RSS:", np.sum((linear3.predict(x3_train) - y3_train) ** 2))
predicted3 = linear3.predict(x3_test)

Coefficients: [  9.27113959e-01   7.92816724e+01   2.26139645e+04   5.57274819e+02
  -7.52387131e+03  -1.18160499e+04]
Intercept: -1139544.91885
R² Value: 0.778138772315
RSS: 1595477191387.6636


In [142]:
x = sm.add_constant(x3_train)
model = sm.OLS(y3_train, x)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.778
Model:                            OLS   Adj. R-squared:                  0.777
Method:                 Least Squares   F-statistic:                     678.7
Date:                Tue, 21 Aug 2018   Prob (F-statistic):               0.00
Time:                        16:28:53   Log-Likelihood:                -13942.
No. Observations:                1168   AIC:                         2.790e+04
Df Residuals:                    1161   BIC:                         2.793e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         -1.14e+06   9.23e+04    -12.347   

In [129]:
#DECISION TREES:  df1

from sklearn import tree

tree1 = tree.DecisionTreeRegressor()
tree1.fit(x1_train, y1_train)
print("Accuracy: \n", tree1.score(x1_test, y1_test))

Accuracy: 
 0.784510924879


In [130]:
tree1.feature_importances_

array([  2.93484196e-02,   1.36182932e-01,   6.43866400e-01,
         1.30630501e-02,   3.37644485e-02,   4.70673680e-02,
         1.29257223e-03,   7.83808726e-03,   8.00886112e-03,
         7.98075415e-03,   2.45209298e-02,   8.58472232e-03,
         4.72728689e-03,   0.00000000e+00,   5.79404216e-07,
         2.27230576e-04,   2.93387048e-05,   4.56647441e-04,
         1.77323231e-03,   6.12305845e-03,   1.46949234e-04,
         1.80605235e-04,   0.00000000e+00,   2.16266733e-04,
         1.26129180e-03,   0.00000000e+00,   2.16920063e-03,
         7.28798605e-04,   1.26784215e-03,   3.43742645e-03,
         2.16541896e-05,   2.11967751e-03,   8.10556600e-04,
         6.93951709e-04,   1.97106393e-05,   6.29964237e-04,
         2.68609597e-04,   8.51724197e-07,   5.44333643e-03,
         2.99010981e-03,   1.10435122e-03,   1.15072857e-03,
         4.82197942e-04])

In [131]:
#DECISION TREES:  df2

tree2 = tree.DecisionTreeRegressor()
tree2.fit(x2_train, y2_train)
print("Accuracy: \n", tree2.score(x2_test, y2_test))

Accuracy: 
 0.697199840874


In [132]:
tree2.feature_importances_

array([ 0.0656274 ,  0.32441702,  0.11899584,  0.00787188,  0.05135213,
        0.0192407 ,  0.41249502])

In [133]:
#DECISION TREES:  df3

tree3 = tree.DecisionTreeRegressor()
tree3.fit(x3_train, y3_train)
print("Accuracy: \n", tree3.score(x3_test, y3_test))

Accuracy: 
 0.762727366933


In [134]:
tree3.feature_importances_

array([ 0.06222286,  0.19467761,  0.65485682,  0.06645063,  0.00430423,
        0.01748785])

In [136]:
# http://www.graphviz.org/Download.php
# https://pypi.python.org/pypi/graphviz
# Then call
tree.export_graphviz(tree1, out_file='tree1.dot') 
!dot -Tpng tree1.dot -o tree1.png

/bin/sh: dot: command not found
